In [ ]:

def get_daily_prices(symbol):
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={symbol}&outputsize=full&apikey={API_KEY}'
    r = requests.get(url)
    
    data = r.json()

    if r.status_code == 200:
       
        price_data = data["Time Series (Daily)"]

        dates = []
        full_dates = list(price_data.keys())
    
        today = date(2020,2,1)
   
    
        prices = []
        for i in range(0, len(full_dates)):
            day = dt.strptime(full_dates[i], "%Y-%m-%d").date()
            if day < today - relativedelta(years=10):
                break
            else:
                dates.append(full_dates[i])
                prices.append(float(price_data[full_dates[i]]["4. close"]))

        his_prices = {"dates": dates, "prices": prices}
    
        return his_prices
    else: return None

get_daily_prices('IBM')

In [108]:
# A template method for showing one data for all stocks

df_incremental = []
for symbol in symbol_list:
    temp = get_daily_prices(symbol)
    temp2 = dict()
    temp2["date"] = temp["dates"]
    temp2[symbol] = temp["prices"]
    temp_df = pd.DataFrame.from_dict(temp2)
    df_incremental.append(temp_df)

list_of_dates = [df_incremental[i]["date"].tolist() for i in range(len(df_incremental))]
unique_dates = sorted(list(set([item for sublist in list_of_dates for item in sublist])))
df= pd.DataFrame(unique_dates, columns=["date"])

for i in range(0, len(df_incremental)):
    df = pd.merge(df, df_incremental[i], on="date")

#df # also works if there are missing dates for some stocks even first one
df['date'] = pd.to_datetime(df['date'])

# Set the date column as the index
df.set_index('date', inplace=True)

# Resample the data to weekly frequency and calculate the mean price
df_weekly = df.resample('W').mean()  # W for weekly, M for monthly, A for annual

# Print the weekly data
print(df_weekly)


                  AAPL      MSFT      GOOG      AMZN      NVDA     BRK-B  \
date                                                                       
2014-03-30  537.160000   39.8300  559.2250  338.3800   17.8450  123.5000   
2014-04-06  538.310000   40.9280  560.8020  335.5870   18.3940  124.5060   
2014-04-13  524.064000   39.7318  545.7480  321.0950   18.4820  122.8460   
2014-04-20  520.897475   39.8350  540.4000  320.1450   18.4550  124.8100   
2014-04-27  545.465800   39.8780  526.3420  325.1500   18.9320  127.0120   
...                ...       ...       ...       ...       ...       ...   
2023-02-26  148.375000  252.0425   91.0675   94.9225  220.8975  303.2425   
2023-03-05  147.516000  250.4500   91.4480   93.4380  233.2380  306.9320   
2023-03-12  151.478000  253.1260   93.6140   92.8400  234.8480  309.5820   
2023-03-19  153.380000  267.1560   97.1980   96.5000  245.0460  300.3220   
2023-03-26  158.738000  275.3060  104.8620   98.7720  265.0740  300.3860   

           

In [105]:
def get_daily_adjusted_prices(symbol):
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={symbol}&outputsize=full&apikey={API_KEY}'
    
    r = requests.get(url)
    #data_adjusted = r.json()
    data = r.json()
  
    if   r.status_code==200:
       
        #price_data_adj = data_adjusted["Time Series (Daily)"]
        price_data = data["Time Series (Daily)"]
        price_df = pd.DataFrame(price_data).transpose().reset_index()
        price_df.rename(columns={'index':'Date'},inplace=True)
        #price_df_adj = pd.DataFrame(price_data_adj).reset_index()
        #price_df_adj.rename(columns={'index':'Date'},inplace=True)
        #price_df = price_df.merge(price_df_adj.loc[:,["Date","5. adjusted close"]], on='Date',how='left')
        price_df.loc[:,'Date'] = [dt.strptime(d,'%Y-%m-%d') for d in price_df.Date]
        end_date = dt(2020,2,1)
        start_date = dt(2010,1,1)
        price_df = price_df.loc[(price_df.Date>=start_date)&(price_df.Date<=end_date),:].reset_index(drop=True)
        return price_df
    else: return None

def get_monthly_adjusted_prices(symbol):
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY_ADJUSTED&symbol={symbol}&outputsize=full&apikey={API_KEY}'
    
    r = requests.get(url)
    data = r.json()
  
    if   r.status_code==200:
        price_data = data["Monthly Adjusted Time Series"]
        price_df = pd.DataFrame(price_data).transpose().reset_index()
        price_df.rename(columns={'index':'Date'},inplace=True)

        price_df.loc[:,'Date'] = [dt.strptime(d,'%Y-%m-%d') for d in price_df.Date]
        end_date = dt(2020,2,1)
        start_date = dt(2010,1,1)
        price_df = price_df.loc[(price_df.Date>=start_date)&(price_df.Date<=end_date),:].reset_index(drop=True)
        return price_df
    else: return None

def get_weekly_adjusted_prices(symbol):
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_WEEKLY_ADJUSTED&symbol={symbol}&outputsize=full&apikey={API_KEY}'
    
    r = requests.get(url)
    data = r.json()
  
    if   r.status_code==200:
        price_data = data["Weekly Adjusted Time Series"]
        price_df = pd.DataFrame(price_data).transpose().reset_index()
        price_df.rename(columns={'index':'Date'},inplace=True)

        price_df.loc[:,'Date'] = [dt.strptime(d,'%Y-%m-%d') for d in price_df.Date]
        end_date = dt(2020,2,1)
        start_date = dt(2010,1,1)
        price_df = price_df.loc[(price_df.Date>=start_date)&(price_df.Date<=end_date),:].reset_index(drop=True)
        return price_df
    else: return None

In [7]:
get_monthly_adjusted_prices("IBM")

NameError: name 'get_monthly_adjusted_prices' is not defined

In [101]:
os.mkdir("")
if not os.path.isdir("Daily"):
    os.mkdir("Daily")
if not os.path.isdir("Weekly"):
    os.mkdir("Weekly")
if not os.path.isdir("Monthly"):
    os.mkdir("Monthly")
for stock in symbol_list:

    stock_daily = get_daily_adjusted_prices("META")
    stock_daily.to_csv("Daily\\"+stock+'.csv',index=0)
    stock_weekly = get_weekly_adjusted_prices("META")
    stock_weekly.to_csv("Weekly\\"+stock+'.csv',index=0)
    stock_montly = get_monthly_adjusted_prices("META")
    stock_df.to_csv("Monthly\\"+stock+'.csv',index=0)
    

# Fundamental Data Collection

In [ ]:
url = f'https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={}&apikey={API_KEY}'

def 

# Macroeconomic Data

In [7]:
import pandas as pd
from sas7bdat import SAS7BDAT
with SAS7BDAT('g_ecind_mth.sas7bdat') as reader:
    df = reader.to_data_frame()

df['datadate'] = pd.to_timedelta(df.datadate,unit='D') + pd.Timestamp('1960-1-1')


    

[g_ecind_mth.sas7bdat] header length 65536 != 8192
[g_ecind_mth.sas7bdat] [g_ecind_mth.sas7bdat] header length 65536 != 8192
[g_ecind_mth.sas7bdat] [g_ecind_mth.sas7bdat] [g_ecind_mth.sas7bdat] header length 65536 != 8192
[g_ecind_mth.sas7bdat] [g_ecind_mth.sas7bdat] [g_ecind_mth.sas7bdat] [g_ecind_mth.sas7bdat] header length 65536 != 8192
[g_ecind_mth.sas7bdat] [g_ecind_mth.sas7bdat] [g_ecind_mth.sas7bdat] [g_ecind_mth.sas7bdat] [g_ecind_mth.sas7bdat] header length 65536 != 8192


,econiso,auto,bond10yr,bond20yr,bond30yr,cabgdp1,cabgdp2,cpi,cpi1,cpi3,...,tbill3m,tbill6m,txcr,unemp,unemp1,unemp2,wpi1,wpi3,wpir,datadate
0,ARE,NaN,NaN,NaN,NaN,27.2453,NaN,NaN,3.242460e+01,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1981-01-31
1,ARG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.613000e-01,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,1981-01-31
2,AUS,NaN,NaN,NaN,NaN,-4.5334,NaN,NaN,3.001900e+01,NaN,...,NaN,NaN,NaN,NaN,5.7542,NaN,3.820400e+01,NaN,8.4325,1981-01-31
3,AUT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.982330e+01,NaN,...,NaN,NaN,10.9297,NaN,2.4124,NaN,7.958240e+01,NaN,8.0266,1981-01-31
4,BEL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.585350e+01,NaN,...,NaN,NaN,10.0549,NaN,10.7923,NaN,5.590620e+01,NaN,12.2712,1981-01-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27355,UKR,NaN,NaN,NaN,NaN,0.4136,NaN,NaN,3.074059e+02,318.1724,...,NaN,NaN,NaN,NaN,9.3248,NaN,4.046045e+02,NaN,8.2186,2020-12-31
27356,USA,NaN,NaN,NaN,NaN,-2.4011,-2.4239,NaN,1.199278e+02,NaN,...,NaN,NaN,12.3434,NaN,3.5892,3.6406,1.127288e+02,NaN,2.4614,2020-12-31
27357,VEN,NaN,NaN,NaN,NaN,1.8301,NaN,NaN,1.247495e+07,NaN,...,NaN,NaN,NaN,NaN,17.7884,NaN,3.955067e+07,NaN,95.6046,2020-12-31
27358,VNM,NaN,NaN,NaN,NaN,0.7529,NaN,NaN,1.761665e+02,NaN,...,NaN,NaN,NaN,NaN,2.4440,NaN,NaN,NaN,NaN,2020-12-31


In [1]:
import pandas as pd
from sas7bdat import SAS7BDAT
with SAS7BDAT('ecind_mth.sas7bdat') as reader:
    df = reader.to_data_frame()

df['datadate'] = pd.to_timedelta(df.datadate,unit='D') + pd.Timestamp('1960-1-1')


    

[ecind_mth.sas7bdat] header length 65536 != 8192


In [4]:
df

,econiso,auto,bond10yr,bond20yr,bond30yr,cabgdp1,cabgdp2,cpi,cpi1,cpi3,...,tbill3m,tbill6m,txcr,unemp,unemp1,unemp2,wpi1,wpi3,wpir,datadate
0,ARE,NaN,NaN,NaN,NaN,27.2453,NaN,NaN,3.242460e+01,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7701.0
1,ARG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.613000e-01,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,7701.0
2,AUS,NaN,NaN,NaN,NaN,-4.5334,NaN,NaN,3.001900e+01,NaN,...,NaN,NaN,NaN,NaN,5.7542,NaN,3.820400e+01,NaN,8.4325,7701.0
3,AUT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.982330e+01,NaN,...,NaN,NaN,10.9297,NaN,2.4124,NaN,7.958240e+01,NaN,8.0266,7701.0
4,BEL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.585350e+01,NaN,...,NaN,NaN,10.0549,NaN,10.7923,NaN,5.590620e+01,NaN,12.2712,7701.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27355,UKR,NaN,NaN,NaN,NaN,0.4136,NaN,NaN,3.074059e+02,318.1724,...,NaN,NaN,NaN,NaN,9.3248,NaN,4.046045e+02,NaN,8.2186,22280.0
27356,USA,NaN,NaN,NaN,NaN,-2.4011,-2.4239,NaN,1.199278e+02,NaN,...,NaN,NaN,12.3434,NaN,3.5892,3.6406,1.127288e+02,NaN,2.4614,22280.0
27357,VEN,NaN,NaN,NaN,NaN,1.8301,NaN,NaN,1.247495e+07,NaN,...,NaN,NaN,NaN,NaN,17.7884,NaN,3.955067e+07,NaN,95.6046,22280.0
27358,VNM,NaN,NaN,NaN,NaN,0.7529,NaN,NaN,1.761665e+02,NaN,...,NaN,NaN,NaN,NaN,2.4440,NaN,NaN,NaN,NaN,22280.0


In [2]:
df = df.loc[(df.econiso=='USA')&(df.datadate>='2010-01-01'),:]
df

,econiso,auto,bond10yr,bond20yr,bond30yr,cabgdp1,cabgdp2,cpi,cpi1,cpi3,...,tbill3m,tbill6m,txcr,unemp,unemp1,unemp2,wpi1,wpi3,wpir,datadate
924,USA,356.217,NaN,4.38,4.51,-2.8707,-2.7914,2.1749,92.0146,91.7664,...,0.08,0.15,11.4464,9.8,9.6228,9.8371,97.0415,96.4223,6.8318,2010-01-31
925,USA,393.080,NaN,4.40,4.55,-2.8707,-2.7914,2.1728,92.0146,91.6791,...,0.13,0.19,11.4464,9.8,9.6228,9.8371,97.0415,95.9558,6.8318,2010-02-28
926,USA,533.450,NaN,4.55,4.72,-2.8707,-2.7914,2.1735,92.0146,91.7094,...,0.16,0.24,11.4464,9.9,9.6228,9.8371,97.0415,96.4815,6.8318,2010-03-31
927,USA,493.590,NaN,4.36,4.53,-2.8707,-2.9836,2.1740,92.0146,91.7305,...,0.16,0.24,11.4464,9.9,9.6228,9.6576,97.0415,96.3334,6.8318,2010-04-30
928,USA,555.180,NaN,4.05,4.22,-2.8707,-2.9836,2.1729,92.0146,91.6829,...,0.16,0.22,11.4464,9.6,9.6228,9.6576,97.0415,96.0501,6.8318,2010-05-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1147,USA,NaN,NaN,NaN,NaN,-2.2670,-2.2499,NaN,144.4546,NaN,...,NaN,NaN,13.8729,NaN,4.3413,4.3315,136.6738,NaN,1.9844,2028-08-31
1148,USA,NaN,NaN,NaN,NaN,-2.2670,-2.2499,NaN,144.4546,NaN,...,NaN,NaN,13.8729,NaN,4.3413,4.3315,136.6738,NaN,1.9844,2028-09-30
1149,USA,NaN,NaN,NaN,NaN,-2.2670,-2.2208,NaN,144.4546,NaN,...,NaN,NaN,13.8729,NaN,4.3413,4.3146,136.6738,NaN,1.9844,2028-10-31
1150,USA,NaN,NaN,NaN,NaN,-2.2670,-2.2208,NaN,144.4546,NaN,...,NaN,NaN,13.8729,NaN,4.3413,4.3146,136.6738,NaN,1.9844,2028-11-30


In [3]:
df.to_csv("macro_data_1.csv",index=0)

In [ ]:
for stock in symbol_lisst:
    